In [1]:
%%html
<style>

.dataframe td {
    white-space: normal !important;       /* consente il wrapping */
    word-wrap: break-word;               /* forza il wrapping se serve */
    max-width: 900px;                    /* larghezza massima di una cella */
}
</style>

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['PATH'] = './ffmpeg-7.0.2-amd64-static:' + os.environ['PATH']

from time import time
from datetime import datetime
from pathlib import Path
import random
import torch
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 1500)
#pd.set_option('display.max_seq_item_length', 500)
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_info_columns', 500)
pd.set_option('display.max_info_rows', 500)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.width', 21000)

pd.set_option('display.date_dayfirst', True)


import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import HTML
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

from arguments import prepare_finetuning_args, Args
import models
from timm.models import create_model

from dataset.build_dataset import calc_tile_offsets, create_df_video_from_master_df, solve_paths, get_train_test_validation_df, calc_avg_cld_idx
from view_test_tiles import display_video_clip, save_frames_parallel, make_animation_parallel_ffmpeg, filling_missing_tile

from model_analysis import create_df_predictions, get_only_labels
from dataset.data_manager import DataManager, BuildDataset, make_validation_data_builder_from_manos_tracks

input_dir = "../fromgcloud"
output_dir = "../airmassRGB/supervised/"




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Impossibile caricare la libreria torch dynamo


In [2]:
args = prepare_finetuning_args()
args.device='cpu'

In [ ]:
###### carico il dataset
val_m = DataManager(is_train=False, args=args, specify_data_path=args.val_path)
val_m.create_classif_dataloader(args)

Getting dataset...
DATASET length: 2400
Creo il DistributedSampler con world_size 1 e rank 0
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x7fd4a73c9ac0>
Batch_size local: 1


In [ ]:
###### carico il dataset (applico filtro nuvolosità sui video di validation)
# NON SERVE, L'INDICE LO DEVO CALCOLARE PRIMA UNA VOLTA PER TUTTE, E POI DEVO CAMBIARE LA VISUALIZZAZIONE, NON DEVO CANCELLARE LE TILE CLEAR SKY
# Carico il csv di validation, calcolo avg_cloud_idx per ogni video usando
# la funzione `calc_avg_cld_idx` definita in dataset/build_dataset.py
try:
    df_val = pd.read_csv(args.val_path)
except Exception:
    # se non riesco a leggere il csv uso comunque il path originale
    df_val = None

if args.cloudy and df_val is not None and 'path' in df_val.columns:
    print('Calcolo indice di nuvolosità per i video di validation...')
    # Applichiamo direttamente la funzione ai percorsi
    df_val['avg_cloud_idx'] = df_val['path'].apply(lambda p: calc_avg_cld_idx(p))
    # soglia: manteniamo video con avg_cloud_idx >= 0.2 (come in build_dataset.filter_out_clear_sky)
    thresh = 0.2
    kept = df_val[df_val['avg_cloud_idx'] > thresh].copy()
    print(f"Video totali: {len(df_val)} - dopo filtro nuvolosità (> {thresh}): {len(kept)}")
    # salvo temporaneamente il csv filtrato e lo passo a DataManager
    filtered_csv = './val_filtered_for_prediction.csv'
    kept.to_csv(Path(args.csv_folder) / filtered_csv, index=False)
    val_csv_to_use = filtered_csv
else:
    val_csv_to_use = args.val_path

Calcolo indice di nuvolosità per i video di validation...
Video totali: 2400 - dopo filtro nuvolosità (> 0.2): 381


In [ ]:
if val_csv_to_use is not None:
    val_m = DataManager(is_train=False, args=args, specify_data_path=val_csv_to_use)
    val_m.create_classif_dataloader(args)

output/traintest_csv/output/traintest_csv/train_manos_unbalanced_6468.csv output/traintest_csv/output/traintest_csv/test_manos_w_354.csv output/traintest_csv/output/traintest_csv/val_manos_w_2400.csv output/traintest_csv/val_filtered_for_prediction.csv
Getting dataset...
DATASET length: 381
Creo il DistributedSampler con world_size 1 e rank 0
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x7f2db800f9d0>
Batch_size local: 1


### Get predictions

In [4]:
# voglio prendere le predizioni
get_prediction = False
args.test_mode = True

# istanzia l'oggetto del modello 
print(f"Creating model: {args.model} (nb_classes={args.nb_classes})")

args.init_ckpt = './output/checkpoint-best-90_25lug25.pth'

model = create_model(
    args.model,
    num_classes=args.nb_classes,
    drop_rate=0.0,
    drop_path_rate=args.drop_path,
    #attn_drop_rate=0.0,
    drop_block_rate=None,
    **args.__dict__
)
model.to(args.device)
model.eval()  

Creating model: vit_giant_patch14_224 (nb_classes=2)
Caricamento del checkpoint da: ./output/checkpoint-best-90_25lug25.pth
Caricato state_dict con chiave: model
Checkpoint caricato con successo!


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(3, 1408, kernel_size=(2, 14, 14), stride=(2, 14, 14))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1408, out_features=4224, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1408, out_features=1408, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1408, out_features=6144, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=6144, out_features=1408, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
  

In [5]:
#all_paths, all_preds, all_labels = get_path_pred_label(model, val_m.data_loader)
all_paths, all_preds, all_labels = get_only_labels(val_m.data_loader)
all_labels_array = np.array(all_labels)
all_preds_array = np.array(all_preds)
df_predictions = create_df_predictions(all_paths, all_preds, all_labels)

Problema nel caricamento di ./../airmassRGB/supervised/28-02-2023_0000_1065_196/img_00005.png
Problema nel caricamento di ./../airmassRGB/supervised/07-09-2023_0800_426_0/img_00011.pngProblema nel caricamento di ./../airmassRGB/supervised/07-09-2023_0300_852_196/img_00012.pngProblema nel caricamento di ./../airmassRGB/supervised/08-09-2023_0800_426_196/img_00008.png


Problema nel caricamento di ./../airmassRGB/supervised/09-09-2023_0000_213_0/img_00001.png
Problema nel caricamento di ./../airmassRGB/supervised/08-11-2021_1300_426_196/img_00008.png


KeyboardInterrupt: 

### Carico le predizioni

In [7]:
df_predictions = pd.read_csv('output/inference_predictions.csv') #, parse_dates=['time', 'start_time', 'end_time'])
df_predictions

,path,predictions,labels
0,08-11-2021_1300_0_0,0,0
1,09-11-2021_0500_0_0,0,0
2,09-11-2021_0300_0_196,0,0
3,09-11-2021_0100_213_0,1,0
4,08-11-2021_2300_213_196,1,0
...,...,...,...
2395,07-09-2023_1000_1065_196,0,0
2396,08-09-2023_0200_1065_196,0,0
2397,08-09-2023_1800_1065_196,0,0
2398,09-09-2023_1000_1065_196,0,0


### DataBuilder e join tabelle

In [ ]:
#### SOLO SE CLOUDY
# filtro df_predictions per tenere solo i video cloudy già filtrati in df_val
if args.cloudy and df_val is not None and 'path' in df_val.columns:
    # filtro la colonna path di kept per togliere tutto il path "../airmassRGB/supervised/"
    kept['path'] = kept['path'].apply(lambda p: p.replace("../airmassRGB/supervised/", ""))

    df_predictions = df_predictions[df_predictions['path'].isin(kept['path'])]
    print(f"Numero di video nelle predizioni dopo filtro nuvolosità: {len(df_predictions)}")

Numero di video nelle predizioni dopo filtro nuvolosità: 381


In [5]:
# ricreo l'istanza di BuildDataset perché ho bisogno del df_video
val_builder = make_validation_data_builder_from_manos_tracks("medicane_data_input/medicanes_new_windows.csv", input_dir, output_dir)

Cicloni nel train: 12, cicloni nel test: 3, cicloni nella validation: 3
Train rows: 564, Test rows: 176, Validation rows: 206
23 days 08:45:00
7 days 08:05:00
8 days 13:25:00
sorted_metadata_files num :  2437
1)  ->
Con cicloni: 18
Senza cicloni: 198

Creazione delle folder per i 216 video...	
Salvati 0 file - Erano già presenti 3456 file - File totali 3456
216 video per il periodo (effettivo) da 2021-11-08 11:45:00 a 2021-11-09 06:00:00

2)  ->
Con cicloni: 66
Senza cicloni: 714

Creazione delle folder per i 780 video...	
Salvati 0 file - Erano già presenti 12480 file - File totali 12480
780 video per il periodo (effettivo) da 2023-02-27 19:45:00 a 2023-03-02 13:00:00

3)  ->
Con cicloni: 117
Senza cicloni: 1287

Creazione delle folder per i 1404 video...	
Salvati 0 file - Erano già presenti 22464 file - File totali 22464
1404 video per il periodo (effettivo) da 2023-09-05 04:45:00 a 2023-09-10 02:00:00



/media/isacDisk1/VideoMAEv2/dataset/build_dataset.py:1294: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out['neighboring'] = out['neighboring'].fillna(False).astype(bool)


In [6]:
##### ottengo le predictions associate ai video con i rispettivi path
df_filtrato_on_video_path = val_builder.df_video.merge(df_predictions, on='path')

In [8]:
#all(df_filtrato_on_video_path.label == df_filtrato_on_video_path.labels)   # ok
#df_filtrato_on_video_path.label.sum()  # ok
df_filtrato_on_video_path.head(1)  #[['path','orig_paths', 'label']]

# 8-11-2021_1800_426_0  video con label 1 corretta

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths,neighboring,predictions,labels
0,0,0,0,08-11-2021_1300_0_0,0,2021-11-08 11:45:00,2021-11-08 13:00:00,"[../fromgcloud/airmass_rgb_20211108_1145.png, ../fromgcloud/airmass_rgb_20211108_1150.png, ../fromgcloud/airmass_rgb_20211108_1155.png, ../fromgcloud/airmass_rgb_20211108_1200.png, ../fromgcloud/airmass_rgb_20211108_1205.png, ../fromgcloud/airmass_rgb_20211108_1210.png, ../fromgcloud/airmass_rgb_20211108_1215.png, ../fromgcloud/airmass_rgb_20211108_1220.png, ../fromgcloud/airmass_rgb_20211108_1225.png, ../fromgcloud/airmass_rgb_20211108_1230.png, ../fromgcloud/airmass_rgb_20211108_1235.png, ../fromgcloud/airmass_rgb_20211108_1240.png, ../fromgcloud/airmass_rgb_20211108_1245.png, ../fromgcloud/airmass_rgb_20211108_1250.png, ../fromgcloud/airmass_rgb_20211108_1255.png, ../fromgcloud/airmass_rgb_20211108_1300.png]",False,0,0


In [19]:
# Espando dataframe con le associazioni path_offsets -> predictions
records = []
for _, row in df_filtrato_on_video_path.iterrows():
    for orig_path in row['orig_paths']:
        records.append({
            'path': orig_path,
            'predictions': row['predictions'],
            'tmp_label': row['labels'],
            'tile_offset_x': row['tile_offset_x'],
            'tile_offset_y': row['tile_offset_y'],
            'neighboring': row['neighboring'], 
        })

# Li trasformiamo in un nuovo DataFrame
df_mapping = pd.DataFrame(records)
print(df_mapping[['path', 'tile_offset_x', 'tile_offset_y','tmp_label']].duplicated().sum())
#Non ci sono path duplicati se in combinazione con gli offsets, 
# a parte se i video sono generati con un certo overlap in tempo!

# ottengo le predictions per ogni immagine
df_mapping.drop_duplicates(inplace=True)
df_mapping.shape


9472


(29220, 6)

In [30]:
df_mapping.iloc[1000:1200]   #8-11-2021_1800_426_0

,path,predictions,tmp_label,tile_offset_x,tile_offset_y,neighboring
1304,../fromgcloud/airmass_rgb_20211108_2125.png,1,1,426,0,False
1305,../fromgcloud/airmass_rgb_20211108_2130.png,1,1,426,0,False
1306,../fromgcloud/airmass_rgb_20211108_2135.png,1,1,426,0,False
1307,../fromgcloud/airmass_rgb_20211108_2140.png,1,1,426,0,False
1308,../fromgcloud/airmass_rgb_20211108_2145.png,1,1,426,0,False
1309,../fromgcloud/airmass_rgb_20211108_2150.png,1,1,426,0,False
1310,../fromgcloud/airmass_rgb_20211108_2155.png,1,1,426,0,False
1311,../fromgcloud/airmass_rgb_20211108_2200.png,1,1,426,0,False
1316,../fromgcloud/airmass_rgb_20211108_2205.png,1,1,426,0,False
1317,../fromgcloud/airmass_rgb_20211108_2210.png,1,1,426,0,False


In [20]:
# ottengo il dataframe di immagini che serve per creare il video del Mediterraneo, mi servono anche le 'x_pix', 'y_pix', 'source'
df_data_merg = df_mapping.merge(val_builder.master_df, on=['path', 'tile_offset_x', 'tile_offset_y'], how='left').drop(columns='label').rename(columns={'tmp_label':'label'})

In [44]:
### devo risolvere con le tile mancanti
offsets = calc_tile_offsets(stride_x=213, stride_y=196)
df_offsets = pd.DataFrame(offsets, columns=['tile_offset_x', 'tile_offset_y'])
def expand_group(group):
    merged = df_offsets.merge(group, on=['tile_offset_x', 'tile_offset_y'], how='left', indicator=True)
    path_value = group['path'].iloc[0]
    merged['path'] = path_value

    extra_cols = [col for col in group.columns if col not in ['path', 'tile_offset_x', 'tile_offset_y']]
    # Ricopia i valori costanti del gruppo originale
    for col in ['datetime']:
        val = group[col].iloc[0]   #.mode()[0] if not group[col].isnull().all() else None
        merged[col] = merged[col].fillna(val)

    merged[filling_missing_tile] = merged['_merge'] == 'left_only'  # True se mancava
    return merged[['path', 'tile_offset_x', 'tile_offset_y'] + extra_cols + [filling_missing_tile]]

# Applichiamo la funzione a ogni path
expanded_df = df_data_merg.groupby('path', group_keys=False).apply(expand_group).reset_index(drop=True)
expanded_df.predictions = expanded_df.predictions.astype('Int8')
expanded_df.label = expanded_df.label.astype('Int8')

/tmp/ipykernel_16784/4063762423.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  expanded_df = df_data_merg.groupby('path', group_keys=False).apply(expand_group).reset_index(drop=True)


In [31]:
df_data_merg.iloc[1000:1200]

,path,predictions,label,tile_offset_x,tile_offset_y,neighboring,datetime,x_pix,y_pix,source,id_cyc_unico,start_time,end_time
1000,../fromgcloud/airmass_rgb_20211108_2125.png,1,1,426,0,False,2021-11-08 21:25:00,[437],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1001,../fromgcloud/airmass_rgb_20211108_2130.png,1,1,426,0,False,2021-11-08 21:30:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1002,../fromgcloud/airmass_rgb_20211108_2135.png,1,1,426,0,False,2021-11-08 21:35:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1003,../fromgcloud/airmass_rgb_20211108_2140.png,1,1,426,0,False,2021-11-08 21:40:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1004,../fromgcloud/airmass_rgb_20211108_2145.png,1,1,426,0,False,2021-11-08 21:45:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1005,../fromgcloud/airmass_rgb_20211108_2150.png,1,1,426,0,False,2021-11-08 21:50:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1006,../fromgcloud/airmass_rgb_20211108_2155.png,1,1,426,0,False,2021-11-08 21:55:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1007,../fromgcloud/airmass_rgb_20211108_2200.png,1,1,426,0,False,2021-11-08 22:00:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1008,../fromgcloud/airmass_rgb_20211108_2205.png,1,1,426,0,False,2021-11-08 22:05:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1009,../fromgcloud/airmass_rgb_20211108_2210.png,1,1,426,0,False,2021-11-08 22:10:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00


In [ ]:
make_animation_parallel_ffmpeg(df_data_merg, nomefile='validation_prova_neighboring')

In [21]:
grouped = df_data_merg.groupby("path", dropna=False)
print(f" abbiamo {len(list(grouped))} gruppi", flush=True)

 abbiamo 2410 gruppi


In [22]:
list_grouped_df = list(grouped)

In [25]:
list_grouped_df[9]

('../fromgcloud/airmass_rgb_20211108_1230.png',
                          path                      predictions  label  tile_offset_x  tile_offset_y  neighboring       datetime      x_pix  y_pix    source   id_cyc_unico      start_time           end_time     
 9     ../fromgcloud/airmass_rgb_20211108_1230.png       0         0           0              0          False    2021-11-08 12:30:00     []     []        []        0                      NaT                 NaT
 229   ../fromgcloud/airmass_rgb_20211108_1230.png       0         0           0            196          False    2021-11-08 12:30:00     []     []        []        0                      NaT                 NaT
 449   ../fromgcloud/airmass_rgb_20211108_1230.png       0         0         213              0           True    2021-11-08 12:30:00     []     []        []        0                      NaT                 NaT
 673   ../fromgcloud/airmass_rgb_20211108_1230.png       1         0         213            196         

In [29]:
from view_test_tiles import render_and_save_frame

render_and_save_frame((900, list_grouped_df, "./"), overwrite=True)

'./airmass_rgb_20230302_0425.png'